# DataScientest : Docker - Tests CI/CD

#### 1- Authentication
Dans ce premier test, nous allons vérifier que la logique d'identification fonctionne bien. Pour cela, il va falloir effectuer requêtes de type GET sur le point d'entrée /permissions. Nous savons que deux utilisateurs existent alice et bob et leurs mots de passes sont wonderland et builder. Nous allons essayer un 3e test avec un mot de passe qui ne fonctionne pas: clementine et mandarine.

Les deux premières requêtes devraient renvoyer un code d'erreur 200 alors que la troisième devrait renvoyer un code d'erreur 403.

In [ ]:
import os
import requests

# définition de l'adresse de l'API
api_address = '0.0.0.0'

# port de l'API
api_port = 8000

# requête
usernames = 'alice', 'bob', 'clementine'
passwords = 'wonderland', 'builder', 'mandarine'
expected_results = 200, 200, 403
for user, passwd, expected_result in zip(usernames, passwords, expected_results):
    r = requests.get(
        url='http//{address}:{port}/permissions'.format(address=api_address, port=api_port),
        params= {
            'username': user,
            'password': passwd
        }
    )

    output = '''
    ============================
        Authentication test
    ============================

    request done at "/permissions"
    | username = {user}
    | password = {passwd}
    expected result = {expected_result}
    actual restult = {status_code}

    ==>  {test_status}

    '''

    # statut de la requête
    status_code = r.status_code

    # affichage des résultats
    if status_code == expected_result:
        test_status = 'SUCCESS'
    else:
        test_status = 'FAILURE'
    print(output.format(user=user, passwd=passwd, expected_result=expected_result, status_code=status_code, test_status=test_status))

    # impression dans un fichier
    if os.environ.get('LOG') == 1:
        with open('api_test.log', 'a') as file:
            file.write(output)

#### 2- Authorization
Dans ce deuxième test, nous allons vérifier que la logique de gestion des droits de nos utilisateurs fonctionne correctement. 

Nous savons que bob a accès uniquement à la v1 alors que alice a accès aux deux versions. 

Pour chacun des utilisateurs, nous allons faire une requête sur les points d'entrée /v1/sentiment et /v2/sentiment: on doit alors fournir les arguments username, password et sentence qui contient la phrase à analyser.

In [37]:
import os
import requests

# définition de l'adresse de l'API
api_address = '0.0.0.0'

# port de l'API
api_port = 8000

# requête
usernames = 'alice', 'bob'
passwords = 'wonderland', 'builder'
sentence = "I love it, it's fantastic !"
endpoints = '/v1/sentiment', '/v2/sentiment'
expected_results = 200, 200, 200, 403

count=0
for endpoint in endpoints:
    for user, passwd in zip(usernames, passwords):
        expected_result = expected_results[count]
#         print(endpoint, user, passwd, expected_result)
        count+=1
        r = requests.get(
            url='http//{address}:{port}{end}'.format(address=api_address, port=api_port, end=endpoint),
            params= {
                'username': user,
                'password': passwd,
                'sentence': sentence
            }
        )

        output = '''
        ============================
            Authentication test
        ============================

        request done at {endpoint}
        | username = {user}
        | password = {passwd}
        expected result = {expected_result}
        actual restult = {status_code}

        ==>  {test_status}

        '''

        # statut de la requête
        status_code = r.status_code

        # affichage des résultats
        if status_code == expected_result:
            test_status = 'SUCCESS'
        else:
            test_status = 'FAILURE'
        print(output.format(endpoint=endpoint, user=user, passwd=passwd, expected_result=expected_result, status_code=status_code, test_status=test_status))

        # impression dans un fichier
        if os.environ.get('LOG') == 1:
            with open('api_test.log', 'a') as file:
                file.write(output)

MissingSchema: Invalid URL 'http//0.0.0.0:8000/v1/sentiment': No schema supplied. Perhaps you meant http://http//0.0.0.0:8000/v1/sentiment?

#### 3- Content
Dans ce dernier test, nous vérifions que l'API fonctionne comme elle doit fonctionner. Nous allons tester les phrases suivantes avec le compte d'alice:

- life is beautiful
- that sucks

Pour chacune des versions du modèle, on devrait récupérer un score positif pour la première phrase et un score négatif pour la deuxième phrase. Le test consistera à vérifier la positivité ou négativité du score.

In [ ]:
import json
import os
import requests

# définition de l'adresse de l'API
api_address = '0.0.0.0'

# port de l'API
api_port = 8000

# requête
username = 'alice'
password = 'wonderland'
sentences = "life is beautiful", "that sucks"
endpoint = '/v2/sentiment'

for i, sentence in enumerate(sentences):
    r = requests.get(
        url='http//{address}:{port}{end}'.format(address=api_address, port=api_port, end=endpoint),
        params= {
            'username': user,
            'password': passwd,
            'sentence': sentence
        }
    )
    response = r.json()
    score = float(response['score'])
    
    if i==0:
        expected_result = "superieur a 0"
        if score > 0:
            test_status = 'SUCCESS'
        else:
            test_status = 'FAILURE'
    elif i==1:
        expected_result = "inferieur a 0"
        if score < 0:
            test_status = 'SUCCESS'
        else:
            test_status = 'FAILURE' 

    output = '''
    ============================
        Authentication test
    ============================

    request done at {endpoint}
    | username = {user}
    | password = {passwd}
    expected result = {expected_result}
    actual result = {score}

    ==>  {test_status}

    '''

    print(output.format(endpoint=endpoint, user=user, passwd=passwd, expected_result=expected_result, score=score, test_status=test_status))

    # impression dans un fichier
    if os.environ.get('LOG') == 1:
        with open('api_test.log', 'a') as file:
            file.write(output)

#### docker-compose

In [ ]:
version: '3.9'
services:
    fastapi:
        image: datascientest/fastapi:1.0.0
        container_name: datascientest_fastapi_from_compose
    networks:
        my_network_from_compose:
            external: true
    ports:
        - target: 8000
        published: 8000
        protocol: tcp
        mode: host
  python:
    image: authentication_test:latest
    container_name: authenticaton_test_from_compose
    networks:
      my_network_from_compose:
        external: true
    environment:
      LOG: 1
    volumes:
      - .:/home/jovyan/tests
  python:
    image: authorization_test:latest
    container_name: authorizicaton_test_from_compose
    networks:
      my_network_from_compose:
        external: true
    environment:
      LOG: 1
    volumes:
      - .:/home/jovyan/tests
  python:
    image: content_test:latest
    container_name: content_test_from_compose
    networks:
      my_network_from_compose:
        external: true
    environment:
      LOG: 1
    volumes:
      - .:/home/jovyan/tests
